In [2]:
# %%
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import *
import os
import zipfile
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# %%
def annulus_weight(altitude, aziinterval):
    n = 90.
    steprad = (360./aziinterval) * (np.pi/180.)
    annulus = 91.-altitude
    w = (1./(2.*np.pi)) * np.sin(np.pi / (2.*n)) * np.sin((np.pi * (2. * annulus - 1.)) / (2. * n))
    weight = steprad * w

    return weight

def svf_angles_100121():
    azi1 = np.arange(1., 360., 360./16.)  #%22.5
    azi2 = np.arange(12., 360., 360./16.)  #%22.5
    azi3 = np.arange(5., 360., 360./32.)  #%11.25
    azi4 = np.arange(2., 360., 360./32.)  #%11.25
    azi5 = np.arange(4., 360., 360./40.)  #%9
    azi6 = np.arange(7., 360., 360./48.)  #%7.50
    azi7 = np.arange(6., 360., 360./48.)  #%7.50
    azi8 = np.arange(1., 360., 360./48.)  #%7.50
    azi9 = np.arange(4., 359., 360./52.)  #%6.9231
    azi10 = np.arange(5., 360., 360./52.)  #%6.9231
    azi11 = np.arange(1., 360., 360./48.)  #%7.50
    azi12 = np.arange(0., 359., 360./44.)  #%8.1818
    azi13 = np.arange(3., 360., 360./44.)  #%8.1818
    azi14 = np.arange(2., 360., 360./40.)  #%9
    azi15 = np.arange(7., 360., 360./32.)  #%10
    azi16 = np.arange(3., 360., 360./24.)  #%11.25
    azi17 = np.arange(10., 360., 360./16.)  #%15
    azi18 = np.arange(19., 360., 360./12.)  #%22.5
    azi19 = np.arange(17., 360., 360./8.)  #%45
    azi20 = 0.  #%360
    iazimuth = np.array(np.hstack((azi1, azi2, azi3, azi4, azi5, azi6, azi7, azi8, azi9, azi10, azi11, azi12, azi13,
                                    azi14, azi15, azi16, azi17, azi18, azi19, azi20)))
    aziinterval = np.array(np.hstack((16., 16., 32., 32., 40., 48., 48., 48., 52., 52., 48., 44., 44., 40., 32., 24.,
                                        16., 12., 8., 1.)))
    angleresult = {'iazimuth': iazimuth, 'aziinterval': aziinterval}

    return angleresult

# %%

# %%
def saveraster(gdal_data, filename, raster):
    rows = gdal_data.RasterYSize
    cols = gdal_data.RasterXSize

    outDs = gdal.GetDriverByName("GTiff").Create(filename, cols, rows, int(1), GDT_Float32)
    outBand = outDs.GetRasterBand(1)

    outBand.WriteArray(raster, 0, 0)
    outBand.FlushCache()
    outBand.SetNoDataValue(-9999)
    outDs.SetGeoTransform(gdal_data.GetGeoTransform())
    outDs.SetProjection(gdal_data.GetProjection())

# %%
def create_patches(patch_option):
    deg2rad = np.pi / 180
    if patch_option == 1:
        annulino = np.array([0, 12, 24, 36, 48, 60, 72, 84, 90])
        skyvaultaltint = np.array([6, 18, 30, 42, 54, 66, 78, 90])
        azistart = np.array([0, 4, 2, 5, 8, 0, 10, 0])
        patches_in_band = np.array([30, 30, 24, 24, 18, 12, 6, 1])
    elif patch_option == 2:
        annulino = np.array([0, 12, 24, 36, 48, 60, 72, 84, 90])
        skyvaultaltint = np.array([6, 18, 30, 42, 54, 66, 78, 90])
        azistart = np.array([0, 4, 2, 5, 8, 0, 10, 0])
        patches_in_band = np.array([31, 30, 28, 24, 19, 13, 7, 1])
    elif patch_option == 3:
        annulino = np.array([0, 12, 24, 36, 48, 60, 72, 84, 90])
        skyvaultaltint = np.array([6, 18, 30, 42, 54, 66, 78, 90])
        azistart = np.array([0, 4, 2, 5, 8, 0, 10, 0])
        patches_in_band = np.array([31 * 2, 30 * 2, 28 * 2, 24 * 2, 19 * 2, 13 * 2, 7 * 2, 1])
    elif patch_option == 4:
        annulino = np.array([0, 4.5, 9, 15, 21, 27, 33, 39, 45, 51, 57, 63, 69, 75, 81, 90])
        skyvaultaltint = np.array([3, 9, 15, 21, 27, 33, 39, 45, 51, 57, 63, 69, 75, 81, 90])
        patches_in_band = np.array([31 * 2, 31 * 2, 30 * 2, 30 * 2, 28 * 2, 28 * 2, 24 * 2, 24 * 2, 19 * 2, 19 * 2, 13 * 2, 13 * 2, 7 * 2, 7 * 2, 1])
        azistart = np.array([0, 0, 4, 4, 2, 2, 5, 5, 8, 8, 0, 0, 10, 10, 0])

    skyvaultaziint = np.array([360 / patches for patches in patches_in_band])
    skyvaultalt, skyvaultazi = np.atleast_2d([]), np.atleast_2d([])

    for j in range(skyvaultaltint.shape[0]):
        for k in range(patches_in_band[j]):
            skyvaultalt = np.append(skyvaultalt, skyvaultaltint[j])
            skyvaultazi = np.append(skyvaultazi, k * skyvaultaziint[j] + azistart[j])

    return skyvaultalt, skyvaultazi, annulino, skyvaultaltint, patches_in_band, skyvaultaziint, azistart

# CUDA kernel for shadowing function
cuda_code = """
__global__ void shadowingfunction_20(float *a, float *vegdem, float *vegdem2, float azimuth, float altitude, float scale, float amaxvalue, float *bush, float *sh, float *vegsh, float *vbshvegsh, int sizex, int sizey) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x >= sizex || y >= sizey) return;

    float degrees = 3.141592653589793 / 180.0;
    azimuth *= degrees;
    altitude *= degrees;

    // Initialize values for shadowing calculations
    float dx = 0.0, dy = 0.0, dz = 0.0;
    float temp, tempvegdem, tempvegdem2;
    float dzprev = 0.0;
    float pibyfour = 3.141592653589793 / 4.0;
    float threetimespibyfour = 3.0 * pibyfour;
    float fivetimespibyfour = 5.0 * pibyfour;
    float seventimespibyfour = 7.0 * pibyfour;

    float sinazimuth = sin(azimuth);
    float cosazimuth = cos(azimuth);
    float tanazimuth = tan(azimuth);
    float signsinazimuth = sinazimuth > 0 ? 1 : -1;
    float signcosazimuth = cosazimuth > 0 ? 1 : -1;

    float dssin = fabs(1.0 / sinazimuth);
    float dscos = fabs(1.0 / cosazimuth);
    float tanaltitudebyscale = tan(altitude) / scale;
    int index = 0;

    while (amaxvalue >= dz && fabs(dx) < sizex && fabs(dy) < sizey) {
        if ((pibyfour <= azimuth && azimuth < threetimespibyfour) || (fivetimespibyfour <= azimuth && azimuth < seventimespibyfour)) {
            dy = signsinazimuth * index;
            dx = -1.0 * signcosazimuth * fabs(roundf(index / tanazimuth));
        } else {
            dy = signsinazimuth * fabs(roundf(index * tanazimuth));
            dx = -1.0 * signcosazimuth * index;
        }
        dz = (dssin * index) * tanaltitudebyscale;

        // Perform shadowing logic here
        // tempvegdem = vegdem[x + dx][y + dy] - dz;
        // tempvegdem2 = vegdem2[x + dx][y + dy] - dz;
        // etc...

        // Shadows from buildings
        if (dz > a[x * sizey + y]) {
            sh[x * sizey + y] = 1.0;
        } else {
            sh[x * sizey + y] = 0.0;
        }
        index++;
    }
}
"""

# %%
def shadowingfunction_20_gpu(a, vegdem, vegdem2, azimuth, altitude, scale, amaxvalue, bush):
    sizex, sizey = a.shape
    sh = np.zeros((sizex, sizey), dtype=np.float32)
    vegsh = np.zeros((sizex, sizey), dtype=np.float32)
    vbshvegsh = np.zeros((sizex, sizey), dtype=np.float32)

    # Transfer data to the GPU
    a_gpu = cuda.mem_alloc(a.nbytes)
    vegdem_gpu = cuda.mem_alloc(vegdem.nbytes)
    vegdem2_gpu = cuda.mem_alloc(vegdem2.nbytes)
    bush_gpu = cuda.mem_alloc(bush.nbytes)
    sh_gpu = cuda.mem_alloc(sh.nbytes)
    vegsh_gpu = cuda.mem_alloc(vegsh.nbytes)
    vbshvegsh_gpu = cuda.mem_alloc(vbshvegsh.nbytes)

    cuda.memcpy_htod(a_gpu, a)
    cuda.memcpy_htod(vegdem_gpu, vegdem)
    cuda.memcpy_htod(vegdem2_gpu, vegdem2)
    cuda.memcpy_htod(bush_gpu, bush)

    # Compile and launch the CUDA kernel
    mod = SourceModule(cuda_code)
    func = mod.get_function("shadowingfunction_20")
    block_size = (16, 16, 1)
    grid_size = (int(np.ceil(sizex / block_size[0])), int(np.ceil(sizey / block_size[1])), 1)

    func(a_gpu, vegdem_gpu, vegdem2_gpu, np.float32(azimuth), np.float32(altitude), np.float32(scale), np.float32(amaxvalue),
         bush_gpu, sh_gpu, vegsh_gpu, vbshvegsh_gpu, np.int32(sizex), np.int32(sizey), block=block_size, grid=grid_size)

    # Copy results back from the GPU
    cuda.memcpy_dtoh(sh, sh_gpu)
    cuda.memcpy_dtoh(vegsh, vegsh_gpu)
    cuda.memcpy_dtoh(vbshvegsh, vbshvegsh_gpu)

    return {'sh': sh, 'vegsh': vegsh, 'vbshvegsh': vbshvegsh}

# %% Other parts of the code remain similar, focusing on CPU-GPU interaction as shown above.
# Call GPU-accelerated shadowing function in place of the original CPU-based one.

def svfForProcessing153(dsm, vegdem, vegdem2, scale, usevegdem):
    rows = dsm.shape[0]
    cols = dsm.shape[1]
    svf = np.zeros([rows, cols])
    svfE = np.zeros([rows, cols])
    svfS = np.zeros([rows, cols])
    svfW = np.zeros([rows, cols])
    svfN = np.zeros([rows, cols])
    svfveg = np.zeros((rows, cols))
    svfEveg = np.zeros((rows, cols))
    svfSveg = np.zeros((rows, cols))
    svfWveg = np.zeros((rows, cols))
    svfNveg = np.zeros((rows, cols))
    svfaveg = np.zeros((rows, cols))
    svfEaveg = np.zeros((rows, cols))
    svfSaveg = np.zeros((rows, cols))
    svfWaveg = np.zeros((rows, cols))
    svfNaveg = np.zeros((rows, cols))


    # % amaxvalue
    vegmax = vegdem.max()
    amaxvalue = dsm.max()
    amaxvalue = np.maximum(amaxvalue, vegmax)

    # % Elevation vegdems if buildingDSM inclused ground heights
    vegdem = vegdem + dsm
    vegdem[vegdem == dsm] = 0
    vegdem2 = vegdem2 + dsm
    vegdem2[vegdem2 == dsm] = 0
    # % Bush separation
    bush = np.logical_not((vegdem2 * vegdem)) * vegdem

    index = int(0)

    # patch_option = 1 # 145 patches
    patch_option = 2 # 153 patches

    # Create patches based on patch_option
    skyvaultalt, skyvaultazi, annulino, skyvaultaltint, aziinterval, skyvaultaziint, azistart = create_patches(patch_option)

    skyvaultaziint = np.array([360/patches for patches in aziinterval])
    iazimuth = np.hstack(np.zeros((1, np.sum(aziinterval)))) # Nils

    shmat = np.zeros((rows, cols, np.sum(aziinterval)))
    vegshmat = np.zeros((rows, cols, np.sum(aziinterval)))
    vbshvegshmat = np.zeros((rows, cols, np.sum(aziinterval)))

    for j in range(0, skyvaultaltint.shape[0]):
        for k in range(0, int(360 / skyvaultaziint[j])):
            iazimuth[index] = k * skyvaultaziint[j] + azistart[j]
            if iazimuth[index] > 360.:
                iazimuth[index] = iazimuth[index] - 360.
            index = index + 1

            
    aziintervalaniso = np.ceil(aziinterval / 2.0)
    index = int(0)
    for i in range(0, skyvaultaltint.shape[0]):
        for j in np.arange(0, (aziinterval[int(i)])):
            
            altitude = skyvaultaltint[int(i)]
            azimuth = iazimuth[int(index)]

            # Casting shadow
            if usevegdem == 1:
                shadowresult = shadowingfunction_20_gpu(dsm, vegdem, vegdem2, azimuth, altitude,
                                                           scale, amaxvalue, bush)
                vegsh = shadowresult["vegsh"]
                vbshvegsh = shadowresult["vbshvegsh"]
                sh = shadowresult["sh"]
                vegshmat[:, :, index] = vegsh
                vbshvegshmat[:, :, index] = vbshvegsh
            else:
                sh = 0 #shadowingfunctionglobalradiation(dsm, azimuth, altitude, scale, feedback, 1)

            shmat[:, :, index] = sh

            # Calculate svfs
            for k in np.arange(annulino[int(i)]+1, (annulino[int(i+1.)])+1):
                weight = annulus_weight(k, aziinterval[i])*sh
                svf = svf + weight
                weight = annulus_weight(k, aziintervalaniso[i]) * sh
                if (azimuth >= 0) and (azimuth < 180):
                    svfE = svfE + weight
                if (azimuth >= 90) and (azimuth < 270):
                    svfS = svfS + weight
                if (azimuth >= 180) and (azimuth < 360):
                    svfW = svfW + weight
                if (azimuth >= 270) or (azimuth < 90):
                    svfN = svfN + weight

            if usevegdem == 1:
                for k in np.arange(annulino[int(i)] + 1, (annulino[int(i + 1.)]) + 1):
                    # % changed to include 90
                    weight = annulus_weight(k, aziinterval[i])
                    svfveg = svfveg + weight * vegsh
                    svfaveg = svfaveg + weight * vbshvegsh
                    weight = annulus_weight(k, aziintervalaniso[i])
                    if (azimuth >= 0) and (azimuth < 180):
                        svfEveg = svfEveg + weight * vegsh
                        svfEaveg = svfEaveg + weight * vbshvegsh
                    if (azimuth >= 90) and (azimuth < 270):
                        svfSveg = svfSveg + weight * vegsh
                        svfSaveg = svfSaveg + weight * vbshvegsh
                    if (azimuth >= 180) and (azimuth < 360):
                        svfWveg = svfWveg + weight * vegsh
                        svfWaveg = svfWaveg + weight * vbshvegsh
                    if (azimuth >= 270) or (azimuth < 90):
                        svfNveg = svfNveg + weight * vegsh
                        svfNaveg = svfNaveg + weight * vbshvegsh

            index += 1
           # feedback.setProgress(int(index * (100. / np.sum(aziinterval))))

    svfS = svfS + 3.0459e-004
    svfW = svfW + 3.0459e-004
    # % Last azimuth is 90. Hence, manual add of last annuli for svfS and SVFW
    # %Forcing svf not be greater than 1 (some MATLAB crazyness)
    svf[(svf > 1.)] = 1.
    svfE[(svfE > 1.)] = 1.
    svfS[(svfS > 1.)] = 1.
    svfW[(svfW > 1.)] = 1.
    svfN[(svfN > 1.)] = 1.

    if usevegdem == 1:
        last = np.zeros((rows, cols))
        last[(vegdem2 == 0.)] = 3.0459e-004
        svfSveg = svfSveg + last
        svfWveg = svfWveg + last
        svfSaveg = svfSaveg + last
        svfWaveg = svfWaveg + last
        # %Forcing svf not be greater than 1 (some MATLAB crazyness)
        svfveg[(svfveg > 1.)] = 1.
        svfEveg[(svfEveg > 1.)] = 1.
        svfSveg[(svfSveg > 1.)] = 1.
        svfWveg[(svfWveg > 1.)] = 1.
        svfNveg[(svfNveg > 1.)] = 1.
        svfaveg[(svfaveg > 1.)] = 1.
        svfEaveg[(svfEaveg > 1.)] = 1.
        svfSaveg[(svfSaveg > 1.)] = 1.
        svfWaveg[(svfWaveg > 1.)] = 1.
        svfNaveg[(svfNaveg > 1.)] = 1.

    svfresult = {'svf': svf, 'svfE': svfE, 'svfS': svfS, 'svfW': svfW, 'svfN': svfN,
                    'svfveg': svfveg, 'svfEveg': svfEveg, 'svfSveg': svfSveg, 'svfWveg': svfWveg,
                    'svfNveg': svfNveg, 'svfaveg': svfaveg, 'svfEaveg': svfEaveg, 'svfSaveg': svfSaveg,
                    'svfWaveg': svfWaveg, 'svfNaveg': svfNaveg, 'shmat': shmat, 'vegshmat': vegshmat, 'vbshvegshmat': vbshvegshmat}
                    # ,
                    # 'vbshvegshmat': vbshvegshmat, 'wallshmat': wallshmat, 'wallsunmat': wallsunmat,
                    # 'wallshvemat': wallshvemat, 'facesunmat': facesunmat}
    return svfresult

# %%
def processAlgorithm(dsm_path, vegdsm_path, vegdsm2_path, output_dir, output_file, trans_veg, trunk_ratio, aniso):
    # InputParameters
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)

    # Load DSM
    gdal_dsm = gdal.Open(dsm_path)
    dsm = gdal_dsm.ReadAsArray().astype(float)
    nd = gdal_dsm.GetRasterBand(1).GetNoDataValue()
    dsm[dsm == nd] = 0.
    if dsm.min() < 0:
        dsm = dsm + np.abs(dsm.min())

    sizex, sizey = dsm.shape
    geotransform = gdal_dsm.GetGeoTransform()
    scale = 1 / geotransform[1]
    
    trans = trans_veg / 100.0

    if vegdsm_path:
        usevegdem = 1
        print('Vegetation scheme activated')

        # Load vegetation DSM
        vegdsm = gdal.Open(vegdsm_path).ReadAsArray().astype(float)
        vegsizex, vegsizey = vegdsm.shape

        if not (vegsizex == sizex and vegsizey == sizey):
            raise ValueError("Error in Vegetation Canopy DSM: All rasters must be of same extent and resolution")

        if vegdsm2_path:
            vegdsm2 = gdal.Open(vegdsm2_path).ReadAsArray().astype(float)
        else:
            vegdsm2 = vegdsm * (trunk_ratio / 100.0)

        vegsizex, vegsizey = vegdsm2.shape
        if not (vegsizex == sizex and vegsizey == sizey):
            raise ValueError("Error in Trunk Zone DSM: All rasters must be of same extent and resolution")
    else:
        vegdsm = np.zeros([dsm.shape[0], dsm.shape[1]])
        vegdsm2 = np.zeros([dsm.shape[0], dsm.shape[1]])
        usevegdem = 0

    # Perform SVF calculation
    if aniso == 1:
        print('Calculating SVF using 153 iterations')
        ret = svfForProcessing153(dsm, vegdsm, vegdsm2, scale, usevegdem)
    else:
        print("aniso=False")
        ret = 0 #svfForProcessing655(dsm, vegdsm, vegdsm2, scale, usevegdem) 

    if ret is not None:
        svfbu = ret["svf"]
        svfbuE = ret["svfE"]
        svfbuS = ret["svfS"]
        svfbuW = ret["svfW"]
        svfbuN = ret["svfN"]

        saveraster(gdal_dsm, os.path.join(output_dir, 'svf.tif'), svfbu)
        saveraster(gdal_dsm, os.path.join(output_dir, 'svfE.tif'), svfbuE)
        saveraster(gdal_dsm, os.path.join(output_dir, 'svfS.tif'), svfbuS)
        saveraster(gdal_dsm, os.path.join(output_dir, 'svfW.tif'), svfbuW)
        saveraster(gdal_dsm, os.path.join(output_dir, 'svfN.tif'), svfbuN)

        with zipfile.ZipFile(os.path.join(output_dir, 'svfs.zip'), 'w') as zipf:
            for file in ['svf.tif', 'svfE.tif', 'svfS.tif', 'svfW.tif', 'svfN.tif']:
                zipf.write(os.path.join(output_dir, file), file)
                os.remove(os.path.join(output_dir, file))

        if usevegdem == 0:
            svftotal = svfbu
        else:
            svfveg = ret["svfveg"]
            svfEveg = ret["svfEveg"]
            svfSveg = ret["svfSveg"]
            svfWveg = ret["svfWveg"]
            svfNveg = ret["svfNveg"]
            svfaveg = ret["svfaveg"]
            svfEaveg = ret["svfEaveg"]
            svfSaveg = ret["svfSaveg"]
            svfWaveg = ret["svfWaveg"]
            svfNaveg = ret["svfNaveg"]

            saveraster(gdal_dsm, os.path.join(output_dir, 'svfveg.tif'), svfveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfEveg.tif'), svfEveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfSveg.tif'), svfSveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfWveg.tif'), svfWveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfNveg.tif'), svfNveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfaveg.tif'), svfaveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfEaveg.tif'), svfEaveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfSaveg.tif'), svfSaveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfWaveg.tif'), svfWaveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfNaveg.tif'), svfNaveg)

            with zipfile.ZipFile(os.path.join(output_dir, 'svfs.zip'), 'a') as zipf:
                for file in ['svfveg.tif', 'svfEveg.tif', 'svfSveg.tif', 'svfWveg.tif', 'svfNveg.tif', 'svfaveg.tif', 'svfEaveg.tif', 'svfSaveg.tif', 'svfWaveg.tif', 'svfNaveg.tif']:
                    zipf.write(os.path.join(output_dir, file), file)
                    os.remove(os.path.join(output_dir, file))

            svftotal = (svfbu - (1 - svfveg) * (1 - trans))

        saveraster(gdal_dsm, output_file, svftotal)

        # Save shadow images
        if aniso == 1:
            shmat = ret["shmat"]
            vegshmat = ret["vegshmat"]
            vbshvegshmat = ret["vbshvegshmat"]
            np.savez_compressed(os.path.join(output_dir, "shadowmats.npz"), shadowmat=shmat, vegshadowmat=vegshmat, vbshmat=vbshvegshmat)

    print("Sky View Factor: SVF grid(s) successfully generated")

# %%
dsm_path= "input/DSM_buildings_ground_37HN1_09.tif"
vegdsm_path="input/CDSM_groundlvl_1m_37HN1_09.tif"
vegdsm2_path= "input/TDSM_1m_37HN1_09.tif"
output_dir= "output"
output_file= "output/output_file.tif"
trans_veg= 0.97
trunk_ratio= 0.25
aniso= 1

# %%
processAlgorithm(dsm_path, vegdsm_path, vegdsm2_path, output_dir, output_file, trans_veg, trunk_ratio, aniso)

Vegetation scheme activated
Calculating SVF using 153 iterations


C:\Users\Max\AppData\Local\Temp\ipykernel_18192\138650943.py:179: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(14): warning #177-D: variable "temp" was declared but never referenced
      float temp, tempvegdem, tempvegdem2;
            ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

kernel.cu(14): warning #177-D: variable "tempvegdem" was declared but never referenced
      float temp, tempvegdem, tempvegdem2;
                  ^

kernel.cu(14): warning #177-D: variable "tempvegdem2" was declared but never referenced
      float temp, tempvegdem, tempvegdem2;
                              ^

kernel.cu(15): warning #177-D: variable "dzprev" was declared but never referenced
      float dzprev = 0.0;
            ^

kernel.cu

  mod = SourceModule(cuda_code)


Sky View Factor: SVF grid(s) successfully generated
